In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import time
import pickle
import threading
import numpy as np
from progressbar import *
from models import Model
from models.ExpGlm import ExpGlm
from models.WblGlm import WblGlm
from models.NpGlm import NpGlm
from models.RayGlm import RayGlm
from features.delicious.extraction import run as delicious_feature
from features.movielens.extraction import run as movielens_feature
from features.dblp.extraction import run as dblp_feature
from features.utils import timestamp_delta_generator
from features.autoencoder import encode
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
np.random.seed(0)

In [ ]:
def get_model(dist):
    return {
        'np': NpGlm(),
        'wbl': WblGlm(),
        'exp': ExpGlm(),
        'ray': RayGlm(),
    }[dist]

In [ ]:
def generate_c_index(T_true, T_pred, Y):
    total_number_of_pairs = 0
    number_of_correct_predictions = 0

    for i in range(len(T_true)):
        for j in range(len(T_true) - 1, i, -1):
            if Y[i] != 0 or Y[j] != 0:  # if one or both of the samples are in observation window
                total_number_of_pairs += 1
                if T_true[i] > T_true[j] and T_pred[i] > T_pred[j]:
                    number_of_correct_predictions += 1
                if T_true[i] < T_true[j] and T_pred[i] < T_pred[j]:
                    number_of_correct_predictions += 1
                if T_true[i] == T_true[j] and T_pred[i] == T_pred[j]:
                    number_of_correct_predictions += 1

    return number_of_correct_predictions / total_number_of_pairs

In [ ]:
def prepare_data(X, Y, T, convert_to_month=False):
    T = T.astype(np.float64)
    if convert_to_month:
        T /= timestamp_delta_generator(months=1)
    T += np.random.rand(len(T)) * Y

    index = np.argsort(T, axis=0).ravel()
    X = X[index, :]
    Y = Y[index]
    T = T[index]

    return X, Y, T

In [ ]:
def evaluate(model: Model, X_train: np.ndarray, Y_train: np.ndarray, T_train: np.ndarray, X_test: np.ndarray,
             Y_test: np.ndarray, T_test: np.ndarray, acc_thresholds):
    model.fit(X_train, Y_train, T_train)
    T_pred = model.quantile(X_test, .5).ravel()
    c_index = generate_c_index(T_test, np.fmin(T_pred, max(T_test)), Y_test)

    k = Y_test.sum()
    T_test = T_test[:k]
    T_pred = T_pred[:k]

    res = np.abs(T_pred - T_test)

    distance = np.zeros((len(acc_thresholds)))
    for i in range(len(acc_thresholds)):
        distance[i] = (res <= acc_thresholds[i]).sum() / len(res)

    mae = mean_absolute_error(T_test, T_pred)
    rmse = mean_squared_error(T_test, T_pred) ** .5
    msle = mean_squared_log_error(T_test, T_pred)
    mre = (res / T_test).mean()
    mad = median_absolute_error(T_test, T_pred)

    return (mae, mre, rmse, msle, mad, c_index) + tuple(distance)

In [ ]:
def cross_validate(dists, X_stat, X, Y, T, cv, acc_thresholds):
    threads = []
    results = {dist+pos: [] for dist in dists for pos in ['', '_stat']}
    k_fold = StratifiedKFold(n_splits=cv, shuffle=True)

    widget = [Bar('=', '[', ']'), ' ', Percentage()]
    bar = ProgressBar(maxval=cv*len(dists)*2, widgets=widget)
    
    for training_indices, test_indices in k_fold.split(X=X, y=Y):
        X_stat_train = X_stat[training_indices, :]
        X_train = X[training_indices, :]
        Y_train = Y[training_indices]
        T_train = T[training_indices]

        X_stat_test = X_stat[test_indices, :]
        X_test = X[test_indices, :]
        Y_test = Y[test_indices]
        T_test = T[test_indices]

        def worker():
            for dist in dists:
                model = get_model(dist)
                scores = evaluate(model, X_train, Y_train, T_train, X_test, Y_test, T_test, acc_thresholds)
                results[dist].append(scores)
                bar.update(bar.currval+1)
                scores_stat = evaluate(model, X_stat_train, Y_train, T_train, X_stat_test, Y_test, T_test, acc_thresholds)
                results[dist+'_stat'].append(scores_stat)
                bar.update(bar.currval+1)

        job = threading.Thread(target=worker)
        threads.append(job)
        
    bar.start()

    for t in threads:
        t.start()
    for t in threads:
        t.join()
    
    bar.finish()
    
    return results

In [ ]:
def get_name(dist):
    return {
        'np': 'NP-Glm',
        'wbl': 'Wbl-Glm',
        'exp': 'Exp-Glm',
        'ray': 'Ray-Glm',
    }[dist]

In [ ]:
X_list, Y_raw, T_raw = dblp_feature(delta=1, observation_window=6, n_snapshots=3)
# X_list, Y_raw, T_raw = delicious_feature(delta=1, observation_window=6, n_snapshots=9)
# X_list, Y_raw, T_raw = movielens_feature(delta=1, observation_window=6, n_snapshots=9)

In [ ]:
limit = 4000
if len(Y_raw) > limit:
    X = np.stack(X_list, axis=1)  # X.shape = (n_samples, timesteps, n_features)
    X, _, Y_raw, _, T_raw, _ = train_test_split(X, Y_raw, T_raw, train_size=limit, stratify=Y_raw, shuffle=True)
    for i in range(len(X_list)):
        X_list[i] = X[:,i,:]

In [ ]:
X_raw = encode(X_list, epochs=100, latent_factor=2)

In [ ]:
start_time = time.time()
X, Y, T = prepare_data(X_raw, Y_raw, T_raw)
scaler = MinMaxScaler(copy=True)
X_stat = scaler.fit_transform(X_list[0])

dists = [
    'np',
    'wbl',
    'exp',
    'ray',
]

print(len(T))

results = cross_validate(dists, X_stat, X, Y, T, cv=5, acc_thresholds=[i/2 for i in range(1,7)])
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
from tabulate import tabulate
table = []
row = []
header = ['MAE', 'MRE', 'RMSE', 'MSLE', 'MDAE', 'CI', 'ACC-1', 'ACC-2', 'ACC-3', 'ACC-4', 'ACC-5', 'ACC-6']
for pos in ['', '_stat']:
    for dist in dists:
        row.append(get_name(dist)+pos)
        result = np.array(results[dist+pos])
        mean = result.mean(axis=0)
        table.append(mean)
print(tabulate(table, showindex=row, floatfmt=".2f", headers=header))